In [1]:
!pip uninstall mex -y

In [2]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

In [4]:
mat = sio.loadmat('../data/AGG2.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [5]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
A.shape

(516, 758)

# Prueba de paquetería `mex`

In [6]:
A = A.toarray()
b = np.array(b.T.tolist()[0])
c = np.array(c.T.tolist()[0])

minim = Minimizer(A,b,c)

In [7]:
minim.matrix.shape

(517, 1276)

In [8]:
print("Start time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))
minim.solve()
print("End time: ", datetime.datetime.now(pytz.timezone('US/Central')).strftime('%I:%M:%S %p'))

Start time:  05:41:34 PM
End time:  05:41:51 PM


**Valores aproximados**

In [9]:
min_approx = minim.get_min()

In [10]:
coeff_approx = minim.get_coeff()

In [11]:
coeff_approx

array([ 2.05467631e+04,  5.39154344e+02,  5.94588700e+03,  3.76918022e+03,
        2.40486720e+04,  0.00000000e+00,  1.15650580e+04,  1.29835580e+03,
        4.04758400e+04,  2.62522127e+04,  4.77908600e+03,  2.34252500e+03,
        1.66132000e+03,  5.81910379e+03,  4.51772000e+02,  7.62054963e+01,
        0.00000000e+00,  1.01960488e+03,  9.49564443e+02,  7.72235662e+01,
        4.93379381e+02,  2.01855586e+03,  5.11025644e+02,  1.20384177e+03,
        2.06094913e+03,  1.35840800e+03,  2.83686679e+03,  1.07995750e+03,
        1.74563622e+02,  6.92092310e+02,  9.85289223e+02,  1.05930575e+03,
        3.13405070e+02,  9.46828446e+02,  1.42191523e+03,  1.38274515e+03,
        7.50996000e+02,  6.68883909e+02,  6.45916923e+02,  6.77670081e+02,
        1.87568400e+03,  5.50999778e+02,  7.19709687e+02,  1.76290800e+03,
        1.09018900e+03,  9.92073000e+02,  1.08922400e+03,  7.39347000e+02,
        4.41273729e+03,  3.75299000e+03,  6.30360000e+02,  3.44183027e+03,
        1.98794700e+03,  

**Valores objetivo**

In [12]:
min_obj = linprog(c, A_ub=A, b_ub=b).fun

In [13]:
coeff_obj = linprog(c, A_ub=A, b_ub=b).x

In [18]:
coeff_obj

array([1.02731980e+04, 3.46303090e+02, 2.97289849e+03, 1.86209273e+03,
       1.20388873e+04, 4.03399622e+02, 5.81016166e+03, 6.49166593e+02,
       2.02374566e+04, 1.30157287e+04, 1.98609524e+03, 1.14347772e+03,
       8.30640127e+02, 2.90946153e+03, 2.25863388e+02, 2.68909764e+02,
       2.69145819e+02, 5.09790154e+02, 4.74414933e+02, 3.82386186e+01,
       2.45403434e+02, 1.00658214e+03, 2.55541524e+02, 5.83507811e+02,
       1.00138250e+03, 6.78289661e+02, 1.37869571e+03, 5.08915725e+02,
       6.88653437e+01, 3.30426709e+02, 4.79839881e+02, 4.97525824e+02,
       1.50622834e+02, 4.47816230e+02, 6.91271336e+02, 6.79958105e+02,
       3.75116719e+02, 3.18220925e+02, 3.04105240e+02, 3.23190699e+02,
       9.37262054e+02, 2.62448354e+02, 3.50728714e+02, 8.80986918e+02,
       5.44115184e+02, 4.95651723e+02, 5.43854312e+02, 3.69141355e+02,
       2.22253413e+03, 1.87645295e+03, 3.15173615e+02, 1.72087668e+03,
       9.93951353e+02, 9.10388092e+03, 3.81643329e+02, 2.53826567e+03,
      

**Comprobación**

In [21]:
assert min_obj == approx(min_approx, abs=1e+5), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,3) == approx(coeff_approx, abs=1e-3), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -32622709.25641067
El valor aproximado obtenido con mex es:  -32675725.11870891
